In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from copy import deepcopy
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import torch

import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights

import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def create_df(base_dir, labeled=True):
    if labeled:
        dd = {"images": [], "labels": []}
        for i in os.listdir(base_dir):
            img_dirs = os.path.join(base_dir, i)
            for j in os.listdir(img_dirs):
                img = os.path.join(img_dirs, j)
                dd["images"] += [img]
                dd["labels"] += [i]
                
    else:
        dd = {"images": []}
        for i in os.listdir(base_dir):
            img_dirs = os.path.join(base_dir, i)
            dd["images"] += [img_dirs]
            
    return pd.DataFrame(dd)

In [ ]:
train = create_df("/kaggle/input/vehicle-classification/train")
val = create_df("/kaggle/input/vehicle-classification/val")
test = create_df("/kaggle/input/vehicle-classification/test", False)

In [ ]:
le = LabelEncoder()
train["labels"] = le.fit_transform(train.loc[:, "labels"].values)
val["labels"] = le.transform(val.loc[:, "labels"].values)

In [ ]:
EPOCHS = 20
LR = 0.1
BATCH = 64
IM_SIZE= 224
STEP = 5
GAMMA = 0.2
DECAY = 0.9
NUM_CLASSES = train["labels"].nunique()

In [ ]:
class Clod(torch.nn.Module):
    def __init__(self, data, transform=None):
        super(Clod, self).__init__()
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, x):
        image, label = self.data.iloc[x, 0], self.data.iloc[x, -1]
        image = Image.open(image).convert('RGB')
        if self.transform is not None:
            image = self.transform(np.array(image))
            
        return image, label

In [ ]:
transform = transforms.Compose([transforms.ToPILImage(),
                               transforms.ToTensor(),
                               transforms.Resize((IM_SIZE, IM_SIZE)),
                               transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])])

In [ ]:
train_ds = Clod(train, transform)
val_ds = Clod(val, transform)

In [ ]:
train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH, shuffle=False)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
num_ftrs = model.fc.in_features
for param in model.parameters():
    param.requires_grad_ = False
model.fc = torch.nn.Linear(num_ftrs, NUM_CLASSES)
model.fc.requires_grad_ = True

In [ ]:
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=GAMMA, step_size=STEP)

In [ ]:
best_model = deepcopy(model)
best_acc = 0.0

acc_train = []
acc_val = []
loss_train = []
loss_val = []

for i in range(1, EPOCHS+1):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    train_total = 0
    
    for data, label in train_dl:
        optimizer.zero_grad()
        if torch.cuda.is_available():
            data, label = data.cuda(), label.cuda()
            
        out = model(data)
        loss = criterion(out, label)
        train_loss += loss.item()
        train_acc += (out.argmax(1) == label).sum().item()
        train_total += out.size(0)
        loss.backward()
        optimizer.step()
        
    train_loss /= train_total
    train_acc /= train_total
        
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    val_total = 0
    
    with torch.no_grad():
        for data, label in val_dl:
            if torch.cuda.is_available():
                data, label = data.cuda(), label.cuda()
                
            out = model(data)
            loss = criterion(out, label)
            val_loss += loss.item()
            val_acc += (out.argmax(1) == label).sum().item()
            val_total += out.size(0)
            
    val_acc /= val_total
    val_loss /= val_total
    acc_train += [train_acc]
    acc_val += [val_acc]
    loss_train += [train_loss]
    loss_val += [val_loss]
    
    if val_acc > best_acc:
        best_acc = val_acc
        best_model = deepcopy(model)
        
    print("Epoch {} train loss {} acc {} val loss {} acc {}".format(i, train_loss, train_acc, val_loss, val_acc))
    scheduler.step()

In [ ]:
epochs = list(range(1, EPOCHS+1))
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axes[0].plot(epochs, loss_train)
axes[0].plot(epochs, loss_val)
axes[0].set_title("Training and Validation loss")
axes[0].legend(["Training", "Validation"])
axes[1].plot(epochs, acc_train)
axes[1].plot(epochs, acc_val)
axes[1].set_title("Training and Validation accuracies")
axes[1].legend(["Training", "Validation"])
plt.show()

In [ ]:
def predict(value):
    image = plt.imread(value)
    image = transform(image)
    image = image.reshape([1, image.shape[0], image.shape[1], image.shape[2]])
    if torch.cuda.is_available():
        image = image.cuda()
    best_model.eval()
    with torch.no_grad():
        out = model(image)
    index = out.argmax(1).item()
    return index

In [ ]:
predicted = []
for i in test.iloc[:, 0]:
    predicted += [predict(i)]

In [ ]:
predicted = le.inverse_transform(predicted)

In [ ]:
index = 0
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(12, 12))
for i in range(5):
    for j in range(5):
        im = plt.imread(test.iloc[index, 0])
        axes[i][j].imshow(im)
        axes[i][j].set_title("Predicted value is {}".format(predicted[index]))
        index += 1
        
plt.tight_layout()
plt.show()